In [1]:
# author: Kim Iheon
# version: 2
# 빠진 부분 작성

import random
import Prototype_Network as net
import numpy as np

# import Network as net


# 염색체를 생성하고 진화시키는 클래스
class Chrom:
    def __init__(self):
        pass
    
    # 첫번째 세대의 염색체를 생성하는 함수
    # param {Int} num_net 신경망 염색체의 개수
    # param {Int} num_node 신경망의 초기 노드 수
    # param {Int} len_chrom 염색체의 길이
    # param {Int} num_in 입력 노드의 개수
    # param {Int} num_out 출력 노드의 개수
    # return {List} chrom_list 첫번째 세대의 모든 염색체를 리턴
    def create_chrom(self, num_net, num_node, num_in, num_out):
        self.NUM_NET = num_net
        self.NUM_NODE = num_node
        self.NUM_IN = num_in
        self.NUM_OUT = num_out
        self.chrom_list = []
        self.net_list = []
        
        # 현재 세대 수
        self.generation = 1
        
        # 하나의 신경망 염색체
        _network = ''
        
        # 한 세대의 모든 염색체
        chrom_list = []
        
        # 존재하는 모든 노드의 리스트
        _node_list = list(range(num_node))
        
        # 시냅스의 시작과 끝에 존재가능한 노드
        _start_list = _node_list[:-num_out]
        _end_list = _node_list
        
        for m in range(self.NUM_NET):
            for i in range(num_node):
                for j in range(num_node):
                    
                    # 시이클을 제거하기 위해 시작 노드보다 커야함
                    if i > j:
                        
                        # 시냅스의 연결 여부
                        connect = random.randint(0, 1)
                        _network += str(connect)
                    else:
                        _network += '0'
            chrom_list.append(_network)
            _network = ''
        self.chrom_list = chrom_list
        
        return chrom_list
    
    # 현재 세대의 염색체를 출력하는 함수
    # return {List} self.chrom_list 현재 세대의 모든 염색체를 리턴
    def get_chrom(self):
        return self.chrom_list
    
    # 현재 세대를 출력하는 함수
    # return {Int} self.generation 현재 세대를 받아옴
    def get_generation(self):
        return self.generation
    
    # 현재 개체의 품질을 평가하는 함수
    # param {Function} qual_func 개체의 품질을 평가하는 함수
    # return {List} qual_list 각 개체의 품질
    def get_quality(self, qual_func):
        chrom_list = self.expression()
        
        qual_list = []
        
        # 각 개체들의 품질을 평가한다
        for i in range(self.NUM_NET):
            i_qual = qual_func(chrom_list[i])
            qual_list.append(i_qual)
        
        self.qual_list = qual_list
        
        return qual_list
    
    # 한 세대의 염색체들을 신경망으로 발현하는 함수
    # return {List} result_exp 한 세대를 신경망으로 모두 발현
    def expression(self):
        num_node = self.NUM_NODE
        result_exp = []
        
        # 현재 새대의 모든 염색체를 받아옴
        chrom_list = self.chrom_list
        
        for chrom in chrom_list:
            network = {}
            
            # 존재하는 유전자들을 발현
            for i in range(num_node):
                for j in range(num_node):
                    if chrom[j * num_node + i] == '1':
                        network[(i, j)] = random.random()
            
            result_exp.append(network)
            network = {}
        
        self.net_list = result_exp
        
        return result_exp
             
    # 현재 세대 모든 염색체의 적합도 리스트를 리턴
    # return {List} fit_list 현재 세대의 모든 염색체의 적합도
    def fitness(self):
        
        # 현재 새대의 모든 개체를 받아옴
        chrom_list = self.net_list
        
        # 각 개체의 품질, 먼저 get_quality를 실행했어야 함
        qual_list = self.qual_list
        
        # 적합도 리스트
        fit_list = []
        
        # 가장 좋은 품질과 가장 안 좋은 품질
        best_qual = min(qual_list)
        werst_qual = max(qual_list)
        
        # 각 염색체에 대해 적합도를 계산
        for i_qual in qual_list:
            i_fit = (werst_qual - i_qual) \
                    + ((werst_qual - best_qual) / 3)
            fit_list.append(i_fit)
            
        self.fit_list = fit_list
        
        return fit_list
    
    # 적합도 리스트와 룰렛 휠 선택을 이용해 염색체를 선택
    # param {Tuple} num 선택하는 염색체의 개수. 없으면 1개 선택
    # return {List} sel_list 선택된 염색체 리스트
    def selection(self, *num):
        sel_list = []
        
        # 현재 새대의 모든 염색체를 받아옴
        chrom_list = self.chrom_list
        
        # 이전에 fitness를 실행했어야함
        fit_list = self.fit_list
        
        # 선택하는 염색체수. 입력하지 않으면 2개 선택
        cycle = 2 if num == () else num[0]
        point = random.uniform(0, sum(fit_list))
        sum_fit = 0
        
        for k in range(cycle):
            for i in range(self.NUM_NET):
                sum_fit += fit_list[i]
                if point < sum_fit:
                    sel_list.append(self.chrom_list[i])
                    break
        
        return sel_list
    
    # 두 부모 염색체를 교차시켜 자식염색체 두개를 얻음
    # param {List} sel_list 부모로 선택된 두 염색체
    # return {List} child 교차 후 두 자식 염색체
    def crossover(self, sel_list):
        parent1 = sel_list[0]
        parent2 = sel_list[1]
        cut_chrom = random.randint(0, len(parent1))
        
        # 일점 교차 방식을 이용
        child = [parent1[:cut_chrom] + parent2[cut_chrom:],
                parent2[:cut_chrom] + parent1[cut_chrom:]]
        
        return child
    
    # 일정 확률로 염색체의 유전자를 변이시킴
    # param {List} chrom 변이시킬 염색체
    # return {List} result_mut 변이 후 염색체
    def mutation(self, chrom):
        num_node = self.NUM_NODE
        result_mut = chrom
        
        # 변이 확률
        p_mut = 0.001
        
        for k in range(len(chrom)):
            
            # 시이클을 제거
            if k // num_node > k % num_node:
                point = random.random()
                if point < p_mut:
                    result_mut = \
                            chrom[:k - 1] + str(1 - int(chrom[k])) + chrom[k:]
        
        return result_mut
    
    # 해집단중 가장 품질이 낮은 해를 대치
    # 정규화를 만든 후에 군집대치 적용
    # param {List} child 교차, 변이된 자식 염색체 집단
    # return {List} result_rep 대치 이후 염색체 집단
    def replacement(self, child_list):
        result_rep = self.chrom_list
        
        # 대치 횟수
        rep_time = len(child_list)
        
        # 적합도 리스트
        fit_list = self.fit_list
        fit_list = sorted(fit_list)
        print(fit_list)
        
        # 가장 낮은 품질을 횟수만큼 뽑음
        worst_fit = fit_list[:rep_time]
        
        # 정렬했던 것을 초기화
        fit_list = self.fit_list
        
        # 모든 자식 염색체들을 대치
        for k in range(len(worst_fit)):
            fit = worst_fit[k]
            
            # 대치할 염색체의 위치
            rep_loc = fit_list.index(fit)
            result_rep[rep_loc] = child_list[k]
        
        return result_rep
    
    # 각 연산을 수행해 기존 염색체 집단을 진화시킴
    # param {int} num_evol 진화시킬 횟수
    # param {Int} num_child 생성할 자식 염색체의 수
    # param {Function} qual_func 염색체의 품질을 정하는 함수
    # return {Tuple} all_qual, all_net 각 세대별 모든 개체와 그 품질
    def evolution(self, num_evol, num_child, qual_func):
        all_qual = []
        all_net = []
        
        for k in range(num_evol):
            
            # 각 개체의 품질을 계산
            k_qual = self.get_quality(qual_func)
            all_qual.append(k_qual)
            all_net.append(self.net_list)
            
            print(self.generation)
            print(sorted(k_qual))
            
            # 적합도를 계산
            self.fitness()
            
            # 자식 염색체 리스트
            child_list = []
            for i in range(num_child):

                # 교차 연산의 결과인 두 자식 중 하나를 선택
                selec = random.randint(0, 1)

                # 부모 염색체 선택
                sel_list = self.selection()

                # 교차후 자식 염색체
                child = self.crossover(sel_list)[selec]

                # 자식 염색체를 변이
                child = self.mutation(child)
                child_list.append(child)

            # 기존 염색체를 생성된 자식 염색체로 대치
            self.chrom_list = self.replacement(child_list)

            # 세대를 1 증가
            self.generation += 1
        
        # 마지막으로 생성된 세대를 평가함
        last_qual = self.get_quality(qual_func)
        all_qual.append(last_qual)
        all_net.append(self.net_list)
    
        print(self.generation)
        print(sorted(last_qual))
        
        return all_qual, all_net

def quality_chrom(chrom):
    network = net.Network(chrom)
    qual = network.quality(10)
    
    time_calc = np.mean(network.time_calc)
    time_path = np.mean(network.time_path)
    time_train = np.mean(network.time_train)
    time_all = np.mean(network.time_all)
    
    print('culc time: ', time_calc)
    print('path time: ', time_path)
    print('train time: ', time_train)
    print('all time: ', time_all)
    
    return qual

In [2]:
a = Chrom()
a.create_chrom(10, 14, 4, 4)
a.expression()

[{(0, 3): 0.3516689819397354,
  (0, 6): 0.9942884850686109,
  (0, 7): 0.6414967498952505,
  (0, 8): 0.5423211984721041,
  (0, 9): 0.13743935491829018,
  (1, 2): 0.9992516219377424,
  (1, 5): 0.23833196511782095,
  (1, 8): 0.4743423540784153,
  (1, 9): 0.32393878869288495,
  (1, 11): 0.10882225071582774,
  (2, 3): 0.5454364999946395,
  (2, 6): 0.9441646125162807,
  (2, 7): 0.07852691731119965,
  (2, 9): 0.728288841138647,
  (2, 10): 0.7397979572268968,
  (2, 11): 0.5812310971987391,
  (3, 5): 0.23606952666859893,
  (3, 6): 0.5013297656681001,
  (3, 11): 0.1197565656333558,
  (3, 12): 0.8029035414553344,
  (4, 8): 0.49533777062305906,
  (4, 10): 0.9037228017736109,
  (5, 6): 0.6457970059864645,
  (5, 7): 0.2888317781512002,
  (5, 9): 0.43003345625178635,
  (5, 10): 0.07872992720492467,
  (5, 11): 0.21536356502858867,
  (6, 8): 0.2623186439924179,
  (6, 9): 0.8617864367151563,
  (6, 12): 0.6946720784379774,
  (7, 8): 0.8159631566201306,
  (7, 9): 0.48428176381088917,
  (7, 13): 0.87111001

In [ ]:
fit__ = a.evolution(80, 7, quality_chrom)

culc time:  0.0012512296029370553
path time:  0.015004503198425547
train time:  0.04961801201007977
all time:  16.79336246513549
culc time:  0.0010733876718497742
path time:  0.0024119695270741204
train time:  0.10714404141880507
all time:  36.4833679305558
culc time:  0.001549002286705615
path time:  0.001953695316927906
train time:  0.03381310757843375
all time:  10.996392822594508
culc time:  0.0012771571090236218
path time:  0.003090913289696573
train time:  0.1533584445416087
all time:  51.048772076577464
culc time:  0.0014022372067065642
path time:  0.0053489273150404415
train time:  0.20126654023523718
all time:  68.2608652812382
culc time:  0.0010055889346112033
path time:  0.0013789281104834572
train time:  0.0424855123296706
all time:  13.82865573753665
culc time:  0.001154109340597645
path time:  0.0009355362655014687
train time:  0.0256886059530238
all time:  8.031896500848035
culc time:  0.0013512627733495658
path time:  0.0034008770331581673
train time:  0.146521782189736

culc time:  0.0008434778734447625
path time:  0.0017591981996929462
train time:  0.047934928575325934
all time:  16.523123859766997
culc time:  0.0007903544778829738
path time:  0.0010746094019395969
train time:  0.028607610431630827
all time:  9.792919073290477
culc time:  0.0007924509552413962
path time:  0.001503632066942373
train time:  0.04510210998781544
all time:  15.57269907087857
culc time:  0.0008115321068399597
path time:  0.0015051716218295041
train time:  0.04542856650250444
all time:  15.677346730883073
culc time:  0.001181783425206039
path time:  0.0013953500306342903
train time:  0.04987876892914798
all time:  15.98061546277745
culc time:  0.0024903497892423006
path time:  0.0013784149255116063
train time:  0.10743825926762876
all time:  34.94458115122802
culc time:  0.0019165856261971405
path time:  0.00808882206092676
train time:  0.08547516811432265
all time:  29.077982053920323
culc time:  0.0017425488452909638
path time:  0.0015564901225388894
train time:  0.094379

culc time:  0.000852686045312432
path time:  0.000950418630736749
train time:  0.018064713884598512
all time:  5.742537135349949
culc time:  0.0008546017416131752
path time:  0.0011105323524134292
train time:  0.017911719432150985
all time:  5.658034552746358
culc time:  0.0009485845774409282
path time:  0.0009375890053888725
train time:  0.02025908343412595
all time:  6.3959129946140365
culc time:  0.0009082120797643256
path time:  0.0013691775955066987
train time:  0.03596154491343918
all time:  11.660485011148921
culc time:  0.000939988728592053
path time:  0.001840794615645791
train time:  0.03615125667713601
all time:  11.70706938004696
culc time:  0.0009071740464618601
path time:  0.0009386153753894177
train time:  0.019324113986757287
all time:  6.139422615550075
culc time:  0.0008487584305240406
path time:  0.0009109033851473214
train time:  0.017956479856019284
all time:  5.715653425638038
culc time:  0.0008451049031880199
path time:  0.0009586295907411113
train time:  0.01798

culc time:  0.000841623409461983
path time:  0.0009211670853801479
train time:  0.01795338860611348
all time:  5.675942143522207
culc time:  0.0009211874961078014
path time:  0.0022154196694827988
train time:  0.03655451920236324
all time:  11.815500753099059
culc time:  0.0009003976717436176
path time:  0.0009422076705050131
train time:  0.019162403869405206
all time:  6.024961831864857
culc time:  0.0010986416226330675
path time:  0.004985079146081262
train time:  0.04409875540215269
all time:  14.257885729094596
culc time:  0.0010458039778029615
path time:  0.0018079507754009683
train time:  0.04350287733636598
all time:  14.077482724909714
culc time:  0.0008790101034476679
path time:  0.0009216802704941074
train time:  0.018939527534275406
all time:  6.004936326570714
culc time:  0.0008580948986217539
path time:  0.0009375890053888725
train time:  0.01874600410084412
all time:  5.961069785462769
culc time:  0.0008586547368090371
path time:  0.0009427208556189726
train time:  0.0188

culc time:  0.0008425010724460498
path time:  0.0011782727733589127
train time:  0.017764745798642834
all time:  5.625242030878326
culc time:  0.000874350616842631
path time:  0.0009227066407220263
train time:  0.01826199944494026
all time:  5.798016539952641
culc time:  0.0008534091696374576
path time:  0.0009175747904919263
train time:  0.01807684536431588
all time:  5.691454186691772
culc time:  0.0009175018947163691
path time:  0.0009175747904919263
train time:  0.01779688866552216
all time:  5.641967243401268
culc time:  0.0008322956888171327
path time:  0.000922706640267279
train time:  0.01754809136700354
all time:  5.5497843340017425
culc time:  0.000841935402599409
path time:  0.0009216802704941074
train time:  0.017872812145404496
all time:  5.646314433584848
culc time:  0.000824767031495164
path time:  0.0009206539007209358
train time:  0.017495937906160722
all time:  5.538837071458602
culc time:  0.0008622616110161827
path time:  0.000972998770976119
train time:  0.01867976

...

culc time:  0.0008731434657378982
path time:  0.0010361205263507145
train time:  0.018257560463621824
all time:  5.784519261116202
culc time:  0.0008423698603663029
path time:  0.0009391285602760036
train time:  0.018035932138056744
all time:  5.738963828154738
culc time:  0.0008498139587748648
path time:  0.0009504186309641227
train time:  0.017748918697126647
all time:  5.607334440102477
culc time:  0.0008536132773196476
path time:  0.0009370758207296603
train time:  0.017872636531287497
all time:  5.6528637006281315
culc time:  0.0008474084040725219
path time:  0.0028512558919828734
train time:  0.017838732763851
all time:  5.623545954434121
culc time:  0.0008601622177867408
path time:  0.0009534977407383849
train time:  0.018205219960933956
all time:  5.7516133253620865
culc time:  0.0008300271778099697
path time:  0.0009391285607307509
train time:  0.017910018368721553
all time:  5.654074304056849
culc time:  0.0009004414090880305
path time:  0.000940668115617882
train time:  0.01

culc time:  0.0008605033692209541
path time:  0.0009750515109772095
train time:  0.01852355165680881
all time:  5.887868049871031
culc time:  0.0008830456037313442
path time:  0.0012655142245421303
train time:  0.024117994090354436
all time:  7.710922886255503
culc time:  0.0008481781815806822
path time:  0.0009760778802956338
train time:  0.01771371765452391
all time:  5.619805862112116
culc time:  0.0008430696581327043
path time:  0.0009350230802738224
train time:  0.017655190422123934
all time:  5.579646056301499
culc time:  0.0008617396785139549
path time:  0.0011007818375219358
train time:  0.023538133271703888
all time:  7.556491148840905
culc time:  0.0009087456755000031
path time:  0.000940668115617882
train time:  0.017634660780189885
all time:  5.592328910533979
culc time:  0.0008331033719948457
path time:  0.0010135403863387182
train time:  0.017777920099242558
all time:  5.622307125830048
culc time:  0.0008670727204079495
path time:  0.0011649299622149556
train time:  0.023

culc time:  0.0008582960905156059
path time:  0.0009611955156287877
train time:  0.01804513070341644
all time:  5.735753855943585
culc time:  0.0008804301097649676
path time:  0.0009473395202803658
train time:  0.01818783427899826
all time:  5.755216397286858
culc time:  0.0008729160315035435
path time:  0.000940668115617882
train time:  0.017774605986362872
all time:  5.642981296972721
culc time:  0.0008392994977736056
path time:  0.0009550372960802633
train time:  0.017522190733532218
all time:  5.572861237340476
culc time:  0.0008394861105526533
path time:  0.0009457999658479821
train time:  0.01775374877509724
all time:  5.634982282287638
culc time:  0.0008821242033925294
path time:  0.0009381021900480846
train time:  0.01865005404721558
all time:  5.9183979390491
culc time:  0.0009328216334552053
path time:  0.0009565768514221418
train time:  0.018663404755138786
all time:  5.895068035501936
culc time:  0.000869723204314968
path time:  0.0009483658905082848
train time:  0.01868017

culc time:  0.0008338906444629919
path time:  0.000951445000282547
train time:  0.017897384326409985
all time:  5.659746537925457
culc time:  0.0008886877229770107
path time:  0.001098729097066098
train time:  0.023132993228770257
all time:  7.352673565584155
culc time:  0.0008481694340761434
path time:  0.0010607534077280434
train time:  0.017867611939668775
all time:  5.652613266345725
culc time:  0.000856642818307178
path time:  0.004217354377033189
train time:  0.023105079143173363
all time:  7.398773487836479
culc time:  0.0008693616421892743
path time:  0.0009550372960802633
train time:  0.018043927270782256
all time:  5.688471042252786
culc time:  0.0008714989411373608
path time:  0.0009663273658588878
train time:  0.017974410188316588
all time:  5.698099419331811
culc time:  0.0008456472464217768
path time:  0.0009391285611854983
train time:  0.017815550355456396
all time:  5.657348937578718
culc time:  0.0008468164918492056
path time:  0.0009375890049341251
train time:  0.0178

culc time:  0.0008476387541753121
path time:  0.000950418630054628
train time:  0.018005026433315918
all time:  5.688716857870531
culc time:  0.0010131292551808326
path time:  0.000959655961196404
train time:  0.02025092991328768
all time:  6.470095426751868
culc time:  0.0009249780671532128
path time:  0.0017145511046692263
train time:  0.018683847417801176
all time:  5.914949335809979
culc time:  0.000869516180831521
path time:  0.0009719724012029474
train time:  0.017905651950617274
all time:  5.667127677863391
culc time:  0.0009359969656605409
path time:  0.0009375890058436198
train time:  0.019652304604876058
all time:  6.281136088965468
culc time:  0.0010125023530556311
path time:  0.0010299623063474428
train time:  0.020744685831287843
all time:  6.58827936765374
culc time:  0.0012376272844417972
path time:  0.0009422076700502657
train time:  0.024344462853815196
all time:  7.731944483645748
culc time:  0.0009982964590397553
path time:  0.0009499054458501632
train time:  0.01982